# 11. Gnuplot - a tool for data visualization: Part 2

In the second part of this Gnuplot tutorial, we will explore more advanced features and techniques for data visualization. We will cover the following topics:

1. Function Fitting: Gnuplot provides functionality for fitting functions to data points. We will learn how to fit various types of curves, such as polynomials, exponentials, and custom-defined functions, to our data.

2. Histograms: Gnuplot allows us to create histograms, which are useful for visualizing the distribution of data. We will learn how to plot histograms and customize their appearance, including the number of bins and bar styles.

3. Multiplots: Sometimes we need to display multiple plots together to compare different datasets or visualize different aspects of the same data. We will explore how to create multiplots, which enable us to display multiple graphs in a single figure.

4. Scripting: Gnuplot can be used as a scripting language to automate repetitive tasks and create complex plots. We will learn how to write scripts that contain a series of commands to generate plots automatically, making our workflow more efficient.

By the end of this tutorial, you will have a solid understanding of these advanced features in Gnuplot and be able to create sophisticated and customized visualizations for your data. Let's dive in and explore the powerful capabilities of Gnuplot!

In [ ]:
%load_ext gnuplot_kernel
%gnuplot inline pngcairo size 1024,600

## 11.1 Function fitting

`Gnuplot` offers the ability to fit functions to our data. In the following example, a 3rd degree polynomial will be fitted to artificially generated data from the file `fake_data.dat`.

In [ ]:
%%gnuplot
plot 'files/fake_data.dat' w p pt 7 lc 7 ps 2

Fitting is performed using the `fit` command. We specify the function we want to fit (the model) and which parameters from the model should be fitted. First, we define the function, a third-degree polynomial with the general formula: $Ax^3+Bx^2+Cx+D$. Then, we define the fitting of this function and specify the parameters using `via`.

In [ ]:
%%gnuplot
f(x) = a*x**3 + b*x**2 + c*x +d

In [ ]:
%%gnuplot
fit f(x) 'files/fake_data.dat' via a,b,c,d    # fit f(x) to data from file

We receive a full description of what has been done. In this case, 4 iterations were performed to obtain the solution. Then, the obtained parameters (a, b, c, d) are displayed along with their errors:

$a = -0.0138867\pm0.01161$<br>$b = -0.231205\pm0.1762$<br>$c = 0.789545\pm0.7563$<br>$d = 1.05233\pm0.8707$.

We can plot the fitted function. Additionally, the true function, based on which the artificial data was generated, is marked by a black dashed line. It is worth mentioning that we can specify the name to be displayed in the legend using `title` in the `plot` command. The deviation of the fitting from the true function is not significant.

In [ ]:
%%gnuplot
p 'files/fake_data.dat' w p pt 7 lc 7 ps 2 title "DATA", a*x**3+b*x**2+c*x+d w l lw 4 lc 2 title 'FITTED FUNCTION', 'files/gen_data.dat' w l lc 8 lw 6 dt 2 title "REAL DATA"

## 11.2 Histogram

Histogram in `gnuplot` is also created using the `plot` command (assuming you have the appropriate data for the histogram) and the `w(ith) boxes` modifier - what is inside the parentheses is optional. The data in the file `histogram.dat` is as follows (first column: bin center, second column: frequency):

```
-4.01733 3
-3.63559 0
-3.25386 4
-2.87212 3
-2.49038 14
-2.10864 36
-1.72690 54
-1.34516 104
-0.96342 206
-0.58168 276
-0.19994 331
0.18179 263
0.56353 252
0.94527 179
1.32701 133
1.70875 91
2.09049 36
2.47223 10
2.85397 2
3.23571 0
3.61744 2
```

In [ ]:
%%gnuplot
reset
set xlabel "Magnitude [mag]"                                                           
set ylabel "Number"                                                                    
set style fill solid 0.85 border lt -1                                         # fill the bars of histogram
plot 'files/histogram.dat' using 1:2 w boxes lc rgb "green" title "HISTOGRAM"  # draw histogram

## 11.3 Multiple Plots on One Figure - `multiplot`

Let's try to plot both of these graphs on a single figure. For this purpose, we can use the `set multiplot` command. We need to specify the number of rows and columns on the figure using the `layout` modifier.

In [ ]:
%gnuplot inline pngcairo size 1024,1024

In [ ]:
%%gnuplot
reset
set multiplot layout 3, 1
set tmargin 3
set title "sin(x)/x"
plot sin(x)/x w l lw 4 lc rgb 'blue'

set title "Third Degree Polynomial"
set xlabel "FAKE X DATA"
set ylabel "FAKE Y DATA" 
plot 'files/fake_data.dat' w p pt 7 lc 7 ps 0.5 title "DATA", 'files/gen_data.dat' w l lc 8 lw 4 dt 2 title "REAL DATA"

set xlabel "Magnitude [mag]"
set ylabel "Number"
set style fill solid 0.85 border lt -1
plot 'files/histogram.dat' using 1:2 w boxes lc rgb "green" title "HISTOGRAM"
unset multiplot

The code above sets up a `multiplot` with a layout of 3 rows and 1 column. Each plot is defined within a specific layout position using the `set title` command to set the title for each plot.

The first plot shows the function `sin(x)/x` with a solid blue line. The second plot displays two sets of data points: one from the file `fake_data.dat` represented by points with color 7 (cyan) and size 0.5, and another from the file `gen_data.dat` represented by a solid line with color 8 (dark yellow) and line width 4, using dashed line type 2. The third plot is a histogram created from the data in the file `histogram.dat`, with green-colored boxes.

After plotting all the graphs, `unset multiplot` is used to disable the `multiplot` feature.

## 11.4 Script

All commands can be written to a file and executed under gnuplot to automate tasks such as generating plots. Such a gnuplot script can be run in a loop in Bash to automatically generate plots. Let's write all the previous commands into a single file called `gpl_script.gpl`:

```gnuplot
set term pngcairo size 1024,1024 font "Ubuntu Condensed, 16"                           # set the output as a PNG file, specify the size and font for the plot
set output "multiplot.png"                                                             # set the output file name


# FUNCTION DEFINITION AND DATA FITTING
f(x) = a*x**3 + b*x**2 + c*x +d
fit f(x) 'files/fake_data.dat' via a,b,c,d    # fit f(x) to the data from the file

set multiplot layout 3, 1
set tmargin 3
set title "sin(x)/x"                                                                   # title of the first plot
plot sin(x)/x w l lw 4 lc rgb 'blue'

set title "Third Degree Polynomial"                                                    # title of the second plot
set xlabel "FAKE X DATA"                                                               # set the x-axis label
set ylabel "FAKE Y DATA" 
p 'files/fake_data.dat' w p pt 7 lc 7 ps 0.7 title "DATA", a*x**3+b*x**2+c*x+d w l lw 4 lc 2 title 'FITTED FUNCTION', 'files/gen_data.dat' w l lc 8 lw 6 dt 2 title "REAL DATA"


set title "HISTOGRAM"                                                                  # title of the third plot
set xlabel "Magnitude [mag]"                                                           # set the x-axis label
set ylabel "Number"                                                                    # set the y-axis label
set style fill solid 0.85 border lt -1                                                 # fill the histogram bars
plot 'files/histogram.dat' using 1:2 w boxes lc rgb "green" title "HISTOGRAM"          # draw the histogram
unset multiplot
```



You can execute the `gpl_script.gpl` file using `gnuplot gpl_script.gpl` to generate the desired plot.